# Hotel Review Classification

### Team 1: Renetta Nelson, Jacqueline Urenda

### June 26, 2023

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
import time
import random


## Retrieving Data from Website

In [4]:
# Retrieving hotel data

sandiego_hotels = "https://www.tripadvisor.in/Hotels-g60750-San_Diego_California-Hotels.html"

response = requests.get(sandiego_hotels)
time.sleep(5 + 10*random.random())

hotels = response.content

print(hotels)


## Extracting the Data

In [3]:
# Initialize lists

hotel_names = []
hotel_ratings = []
hotel_reviews = []
hotel_prices = []

# Hotel Data

h_content = BeautifulSoup(hotels, 'html.parser')

for hotel_name in BeautifulSoup.find_all('div', {'class': 'listing_title'}):
    hotel_names.append(hotel_name.text)

for hotel_rating in BeautifulSoup.find_all('a', {'class': 'ui_bubble_raing'}):
    hotel_ratings.append(hotel_rating['alt'])

for hotel_review in BeautifulSoup.find_all('a', {'class': 'review_count'}):
    hotel_reviews.append(hotel_review.text)

for hotel_price in BeautifulSoup.find_all('div', {'class': 'price-wrap'}):
    hotel_prices.append(hotel_price.text)


AttributeError: 'str' object has no attribute 'descendants'

## Loading Data 

In [ ]:

hotels_df = pd.DataFrame()

# Load data into dataframe

hotels_df["name"] = hotel_names
hotels_df["review_amt"] = hotel_reviews
hotels_df["ratings"] = hotel_ratings
hotels_df["costs"] = hotel_prices

hotels_df.head(20)


## Data Preparation